In [1]:
from llama_sim import LlamaForCausalLM
#from transformers import LlamaForCausalLM
import torch
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3,4,5"
hf_llm = LlamaForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-70B-Instruct", device_map="auto", torch_dtype=torch.bfloat16)



/home/zhuominc/anaconda3/envs/dgl/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/zhuominc/anaconda3/envs/dgl/lib/python3.9/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/zhuominc/anaconda3/envs/dgl/lib/python3.9/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/zhuominc/anaconda3/envs/dgl/lib/python3.9/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.r

In [9]:
import numpy as np
import random
def setup_seed(seed):
     torch.manual_seed(seed)
     torch.cuda.manual_seed_all(seed)
     np.random.seed(seed)
     random.seed(seed)
     torch.backends.cudnn.deterministic = True
# 设置随机数种子
setup_seed(20)
hf_llm = hf_llm.eval()
hf_llm.config.K = 12
hf_llm.config.L = 150
hf_llm.config.cache_mode = "anns_es"
hf_llm.config.window = 16
hf_llm.set_sparse_attn(sparse=0.05, random_sparse=0.2, window_size=16, kernel_size=5)
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-70B-Instruct", use_fast=False)

/home/zhuominc/anaconda3/envs/dgl/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
input_text = """
System: You are an assistant. Your goal is to be helpful, friendly and answer the questions as more as you can. 
User: Hello, I want to test your mathematic ability, nice to meet you! I have a problem for you.
@<<<assign v0 = v2 - 1>>>@@<<<assign v1 = v2 + 1>>>@@<<<assign v2 = 1>>>@
**Question:**
Can you tell me which variables are equal to 2 in those assignments in '<<<>>>'? The values of those variables can be used before definition as they are constant.
You can solve the problem in a step-by-step way.
Assistant: 
"""
input_tokens = tokenizer.encode(input_text, return_tensors="pt")
print(input_tokens, len(input_tokens[0]))

tensor([[128000,    198,   2374,     25,   1472,    527,    459,  18328,     13,
           4718,   5915,    374,    311,    387,  11190,     11,  11919,    323,
           4320,    279,   4860,    439,    810,    439,    499,    649,     13,
            720,   1502,     25,  22691,     11,    358,   1390,    311,   1296,
            701,  21651,   5845,     11,   6555,    311,   3449,    499,      0,
            358,    617,    264,   3575,    369,    499,    627,     31,  12889,
           7128,    348,     15,    284,    348,     17,    482,    220,     16,
          20761,  19741,  12889,   7128,    348,     16,    284,    348,     17,
            489,    220,     16,  20761,  19741,  12889,   7128,    348,     17,
            284,    220,     16,  20761,  63899,    334,  14924,     25,   1035,
           6854,    499,   3371,    757,    902,   7482,    527,   6273,    311,
            220,     17,    304,   1884,  32272,    304,    364,  12889,   2511,
           5709,     30,    

In [11]:
output = hf_llm.generate(inputs=input_tokens, do_sample=False, max_new_tokens=1024)

/home/zhuominc/anaconda3/envs/dgl/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/zhuominc/anaconda3/envs/dgl/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:394: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/home/zhuominc/anaconda3/envs/dgl/lib/python3.9/site-packages/transformers/generation/utils.py:1636:

In [12]:
#print(output)
output_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(output_text)


System: You are an assistant. Your goal is to be helpful, friendly and answer the questions as more as you can. 
User: Hello, I want to test your mathematic ability, nice to meet you! I have a problem for you.
@<<<assign v0 = v2 - 1>>>@@<<<assign v1 = v2 + 1>>>@@<<<assign v2 = 1>>>@
**Question:**
Can you tell me which variables are equal to 2 in those assignments in '<<<>>>'? The values of those variables can be used before definition as they are constant.
You can solve the problem in a step-by-step way.
Assistant: 
Nice to meet you too! I'm happy to help you with your math problem.

Let's break down the assignments step by step:

1. `v2 = 1` (initial value of v2 is 1)

2. `v1 = v2 + 1` (substitute v2 with its current value, which is 1)
   `v1 = 1 + 1`
   `v1 = 2` (so, v1 is equal to 2)

3. `v0 = v2 - 1` (substitute v2 with its current value, which is 1)
   `v0 = 1 - 1`
   `v0 = 0` (so, v0 is not equal to 2)

From the above steps, we can conclude that `v1` is equal to 2.

So, the answ